In [32]:
import os 
!pwd

/home/mehrajofficial000/venv/ml_project


In [33]:
#os.chdir("../")
!pwd

/home/mehrajofficial000/venv/ml_project


In [34]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [35]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data :Path
    all_params : dict
    params_image_size : list
    params_batch_size : int  
    params_learning_rate :float

In [ ]:
from Classifier.constants import *
from Classifier.utils.utilities import read_yaml, create_directories, save_json 

class ConfigureationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH , params_file_path = PARAMS_FILE_PATH):
        self.config= read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model = "artifacts/training/trained_model.keras",
            training_data ="artifacts/data_ingestion/Chicken-fecal-images",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            params_learning_rate = self.params.LEARNING_RATE
        )
        return evaluation_config    

In [37]:
from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2,  # 20% for validation
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            class_mode = "sparse",
            shuffle = False,
            **dataflow_kwargs
        )
      
    @staticmethod
    def load_model(path : Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluate_model(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            metrics=['accuracy']
        )
        self._valid_generator()
        self.score = self.model.evaluate(
            self.valid_generator,
        )

    def save_score(self):
        score = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"),data=score)
        print(f"Model evaluation score: {score}")
    

In [38]:
try : 
    config = ConfigureationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluate_model()
    evaluation.save_score()
except Exception as e:
    print(f"Error occurred during evaluation: {e}")
    raise e

[2025-08-03 00:52:00,893] INFO: utilities: YAML file config/config.yaml loaded successfully.]
[2025-08-03 00:52:00,896] INFO: utilities: YAML file params.yaml loaded successfully.]
[2025-08-03 00:52:00,898] INFO: utilities: Directory craeted at artifacts]
Found 78 images belonging to 2 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.5000 - loss: 12.3284 
[2025-08-03 00:52:12,564] INFO: utilities: Json file saved at scores.json]
Model evaluation score: {'loss': 12.328390121459961, 'accuracy': 0.5}
